# Lab 3. Custom AI Assistant with OpenVINO™ Toolkit

In [1]:
import argparse
import logging as log
import time
from pathlib import Path
from threading import Thread
from typing import Tuple, List, Optional, Set

import gradio as gr
import librosa
import numpy as np
import openvino as ov
from optimum.intel import OVModelForCausalLM, OVModelForSpeechSeq2Seq
from transformers import AutoConfig, AutoTokenizer, AutoProcessor, PreTrainedTokenizer, TextIteratorStreamer
from transformers.generation.streamers import BaseStreamer
from modelscope import snapshot_download

### Convert and Compress ASR model with optimum-cli

In [2]:
asr_model_id = "BELLE-2/Belle-whisper-large-v3-zh"
ori_asr_model_path = "./model/Qwen2-7B-Instruct-pytorch"
asr_model_path = "./model/Belle-whisper-large-v3-zh-ov"
asr_local_path  = ori_asr_model_path + "/snake7gun/Belle-whisper-large-v3-zh-ov"

if not Path(asr_model_path).exists():
    if not Path(ori_asr_model_path).exists():
        model_dir = snapshot_download(asr_model_id, cache_dir=ori_asr_model_path)
    !optimum-cli export openvino --model {asr_local_path} --task automatic-speech-recognition-with-past --weight-format int8 --trust-remote-code {asr_model_path}

config.json: 100%|█████████████████████████| 1.32k/1.32k [00:00<00:00, 4.71MB/s]
Framework not specified. Using pt to export the model.
pytorch_model.bin: 100%|████████████████████| 6.17G/6.17G [01:01<00:00, 101MB/s]
generation_config.json: 100%|██████████████| 3.87k/3.87k [00:00<00:00, 13.1MB/s]
tokenizer_config.json: 100%|█████████████████| 283k/283k [00:00<00:00, 1.97MB/s]
vocab.json: 100%|██████████████████████████| 1.04M/1.04M [00:00<00:00, 6.10MB/s]
tokenizer.json: 100%|██████████████████████| 2.48M/2.48M [00:00<00:00, 7.11MB/s]
merges.txt: 100%|████████████████████████████| 494k/494k [00:00<00:00, 3.34MB/s]
added_tokens.json: 100%|████████████████████| 34.6k/34.6k [00:00<00:00, 576kB/s]
special_tokens_map.json: 100%|█████████████| 1.75k/1.75k [00:00<00:00, 10.7MB/s]
preprocessor_config.json: 100%|████████████████| 340/340 [00:00<00:00, 1.76MB/s]
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.c

### Prepare system prompt

In [ ]:
TARGET_AUDIO_SAMPLE_RATE = 16000
SYSTEM_CONFIGURATION = (
    "You are Adrishuo - a helpful, respectful, and honest virtual doctor assistant. "
    "Your role is talking to a patient who just came in."
    "Your primary role is to assist in the collection of Symptom information from patients. "
    "Focus solely on gathering symptom details without offering treatment or medical advice."
    "You must only ask follow-up questions based on the patient's initial descriptions to clarify and gather more details about their symtpoms. "
    "You must not attempt to diagnose, treat, or offer health advice. "
    "Ask one and only the symptom related followup questions and keep it short. "
    "You must strictly not suggest or recommend any treatments, including over-the-counter medication. "
    "You must strictly avoid making any assumptions or conclusions about the causes or nature of the patient's symptoms. "
    "You must strictly avoid providing suggestions to manage their symptoms. "
    "Your interactions should be focused solely on understanding and recording the patient's stated symptoms. "
    "Do not collect or use any personal information like age, name, contact, gender, etc. "
    "Ask at most 3 questions then say you know everything and you're ready to summarize the patient. "
    "Remember, your role is to aid in symptom information collection in a supportive, unbiased, and factually accurate manner. "
    "Your responses should consistently encourage the patient to discuss their symptoms in greater detail while remaining neutral and non-diagnostic."
)
GREET_THE_CUSTOMER = "Please introduce yourself and greet the patient"
SUMMARIZE_THE_CUSTOMER = (
    "You are now required to summarize the patient's exact provided symptoms for the doctor's review. "
    "Strictly do not mention any personal data like age, name, gender, contact, non-health information etc. when summarizing."
    "Warn the patients for immediate medical seeking in case they exhibit symptoms indicative of critical conditions such as heart attacks, strokes, severe allergic reactions, breathing difficulties, high fever with severe symptoms, significant burns, or severe injuries."
    "Summarize the health-related concerns mentioned by the patient in this conversation, focusing only on the information explicitly provided, without adding any assumptions or unrelated symptoms."
)


### Initialize ASR

In [ ]:
from notebook_utils import device_widget

asr_device = device_widget(default="GPU", exclude=["NPU"])

asr_device

In [ ]:
asr_model = OVModelForSpeechSeq2Seq.from_pretrained(asr_model_path, device=asr_device)
asr_processor = AutoProcessor.from_pretrained(asr_model_path)

### Initialize LLM

In [ ]:
llm_device = device_widget(default="GPU", exclude=["NPU"])

llm_device

In [ ]:
llm_model_path = "./model/Qwen2-7B-Instruct-ov"
ov_config = {'PERFORMANCE_HINT': 'LATENCY', 'NUM_STREAMS': '1', "CACHE_DIR": ""}

chat_model = OVModelForCausalLM.from_pretrained(llm_model_path, device=llm_device, config=AutoConfig.from_pretrained(llm_model_path), ov_config=ov_config)
chat_tokenizer = AutoTokenizer.from_pretrained(llm_model_path)

### Prepare helper function for chatbot

In [ ]:
def respond(prompt: str, streamer: BaseStreamer | None = None) -> str:
    """
    Respond to the current prompt

    Params:
        prompt: user's prompt
        streamer: if not None will use it to stream tokens
    Returns:
        The chat's response
    """
    start_time = time.time()  # Start time
    # tokenize input text
    inputs = chat_tokenizer(prompt, return_tensors="pt").to(chat_model.device)
    input_length = inputs.input_ids.shape[1]
    # generate response tokens
    outputs = chat_model.generate(**inputs, max_new_tokens=256, do_sample=True, temperature=0.6, top_p=0.9, top_k=50, streamer=streamer)
    tokens = outputs[0, input_length:]
    end_time = time.time()  # End time
    log.info("Chat model response time: {:.2f} seconds".format(end_time - start_time))
    # decode tokens into text
    return chat_tokenizer.decode(tokens, skip_special_tokens=True)

def get_conversation(history: List[List[str]]) -> str:
    """
    Combines all messages into one string

    Params:
        history: history of the messages (conversation) so far
    Returns:
        All messages combined into one string
    """
    # the conversation must be in that format to use chat template
    conversation = [
        {"role": "system", "content": SYSTEM_CONFIGURATION},
        {"role": "user", "content": GREET_THE_CUSTOMER}
    ]
    # add prompts to the conversation
    for user_prompt, assistant_response in history:
        if user_prompt:
            conversation.append({"role": "user", "content": user_prompt})
        if assistant_response:
            conversation.append({"role": "assistant", "content": assistant_response})

    # use a template specific to the model
    return chat_tokenizer.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)


def generate_initial_greeting() -> str:
    """
    Generates customer/patient greeting

    Returns:
        Generated greeting
    """
    conv = get_conversation([[None, None]])
    return respond(conv)


def chat(history: List[List[str]]) -> List[List[str]]:
    """
    Chat function. It generates response based on a prompt

    Params:
        history: history of the messages (conversation) so far
    Returns:
        History with the latest chat's response (yields partial response)
    """
    # convert list of message to conversation string
    conversation = get_conversation(history)

    # use streamer to show response word by word
    chat_streamer = TextIteratorStreamer(chat_tokenizer, skip_prompt=True, skip_special_tokens=True)

    # generate response for the conversation in a new thread to deliver response token by token
    thread = Thread(target=respond, args=[conversation, chat_streamer])
    thread.start()

    # get token by token and merge to the final response
    history[-1][1] = ""
    for partial_text in chat_streamer:
        history[-1][1] += partial_text
        # "return" partial response
        yield history

    # wait for the thread
    thread.join()
    
def prepare_text(message, history: List[List[str]]) -> {str, List[List[str]]}:
    history.append([message, None])
    return "", history

def generate_initial_greeting() -> str:
    """
    Generates customer/patient greeting

    Returns:
        Generated greeting
    """
    conv = get_conversation([[None, None]])
    return respond(conv)

def summarize(conversation: List) -> str:
    """
    Summarize the patient case

    Params
        conversation: history of the messages so far
    Returns:
        Summary
    """
    conversation.append([SUMMARIZE_THE_CUSTOMER, None])
    for partial_summary in chat(conversation):
        yield partial_summary[-1][1]

### Prepare helper function for speech recognition

In [ ]:
def transcribe(audio: Tuple[int, np.ndarray], conversation: List[List[str]]) -> List[List[str]]:
    """
    Transcribe audio to text

    Params:
        audio: audio to transcribe text from
        conversation: conversation history with the chatbot
    Returns:
        User prompt as a text
    """
    start_time = time.time()  # Start time for ASR process

    sample_rate, audio = audio
    # the whisper model requires 16000Hz, not 44100Hz
    audio = librosa.resample(audio.astype(np.float32), orig_sr=sample_rate, target_sr=TARGET_AUDIO_SAMPLE_RATE).astype(np.int16)

    # get input features from the audio
    input_features = asr_processor(audio, sampling_rate=TARGET_AUDIO_SAMPLE_RATE, return_tensors="pt").input_features

    # use streamer to show transcription word by word
    text_streamer = TextIteratorStreamer(asr_processor, skip_prompt=True, skip_special_tokens=True)

    # transcribe in the background to deliver response token by token
    thread = Thread(target=asr_model.generate, kwargs={"input_features": input_features, "streamer": text_streamer})
    thread.start()

    conversation.append(["", None])
    # get token by token and merge to the final response
    for partial_text in text_streamer:
        conversation[-1][0] += partial_text
        # "return" partial response
        yield conversation

    end_time = time.time()  # End time for ASR process
    log.info(f"ASR model response time: {end_time - start_time:.2f} seconds")  # Print the ASR processing time

    # wait for the thread
    thread.join()

    return conversation

### Build Gradio interface

In [ ]:
initial_message = generate_initial_greeting()

with gr.Blocks(title="Talk to Adrishuo - a custom AI assistant working as a healthcare assistant") as demo:
    gr.Markdown("""
    # Talk to Adrishuo - a custom AI assistant working today as a healthcare assistant

    Instructions for use:
    - record your question/comment using the first audio widget ("Your voice input")
    - wait for the chatbot to response ("Chatbot")
    - click summarize button to make a summary
    """)
    with gr.Row():
        # user's input
        input_audio_ui = gr.Audio(sources=["microphone"], scale=5, label="Your voice input")
        # submit button
        submit_audio_btn = gr.Button("Submit", variant="primary", scale=1, interactive=False)
        input_audio = gr.Checkbox(
                value=True,
                label="Input by audio",
                interactive=True,
                info="Input method",
            )

    # chatbot
    chatbot_ui = gr.Chatbot(value=[[None, initial_message]], label="Chatbot")
    with gr.Row():
        msg = gr.Textbox(
            label="QA Message Box",
            placeholder="Chat Message Box",
            show_label=False,
            container=False,
        )
        submit_text_btn = gr.Button("Submit", variant="primary")
    # summarize
    summarize_button = gr.Button("Summarize", variant="primary", interactive=False)
    summary_ui = gr.Textbox(label="Summary", interactive=False)

    # events
    # block submit button when no audio input
    input_audio_ui.change(lambda x: gr.Button(interactive=False) if x is None else gr.Button(interactive=True), inputs=input_audio_ui, outputs=submit_audio_btn)
    if input_audio:
    # block buttons, do the transcription and conversation, clear audio, unblock buttons
        submit_audio_btn.click(lambda: gr.Button(interactive=False), outputs=submit_audio_btn) \
            .then(lambda: gr.Button(interactive=False), outputs=summarize_button)\
            .then(transcribe, inputs=[input_audio_ui, chatbot_ui], outputs=chatbot_ui)\
            .then(chat, chatbot_ui, chatbot_ui)\
            .then(lambda: None, inputs=[], outputs=[input_audio_ui])\
            .then(lambda: gr.Button(interactive=True), outputs=summarize_button)
    else:
        submit_text_btn.click(lambda: gr.Button(interactive=False), outputs=submit_text_btn) \
            .then(prepare_text, [msg, chatbot_ui], [msg, chatbot_ui])\
            .then(chat, [chatbot_ui], chatbot_ui)
    # block button, do the summarization, unblock button
    summarize_button.click(lambda: gr.Button(interactive=False), outputs=summarize_button) \
        .then(summarize, inputs=chatbot_ui, outputs=summary_ui) \
        .then(lambda: gr.Button(interactive=True), outputs=summarize_button)

demo.queue().launch()